##Getting things started

In [0]:
#<--------Dependencies--------->
import pandas as pd
import emmettsmath as em #<-- The source for this can be found here:
#https://colab.research.google.com/drive/1PA10KpZsJujB5gwCD0i1BSRN3eb2TVZq
import scipy.stats
import numpy as np
import matplotlib.pyplot as plt
!pip install plotly==4.0.0
import plotly.graph_objects as go
#<--------Dependencies--------->

     |████████████████████████████████| 6.8MB 5.1MB/s 
  Found existing installation: plotly 3.6.1
    Uninstalling plotly-3.6.1:
      Successfully uninstalled plotly-3.6.1


In [0]:
import plotly.express as px

In [0]:
#<--------Reading in Data---------->
hist_earnamd = pd.read_csv('HistoricalQuotes.csv')
satisfaccens = pd.read_csv('http://opendata.emmettboudreau.com/Technology/Small%281-50%20rows%29/AMD%20and%20Nvidea%20US%20Census%20-%20CPU%20data.csv')
searchqueries = pd.read_csv('http://opendata.emmettboudreau.com/Technology/Small%281-50%20rows%29/Ryzen%20Vs%20Core%20I9%20Google%20Search%20Queries.csv')
#<--------Reading in Data---------->

In [0]:
hist_earnamd.head(5)

,date,close,volume,open,high,low
0,16:00,30.45,"119,109,070",32.05,32.30,30.300
1,2019/07/30,33.87,72198880.0000,33.21,33.94,32.940
2,2019/07/29,33.48,45921980.0000,34.14,34.18,32.980
3,2019/07/26,34.02,43502780.0000,34.23,34.77,33.865
4,2019/07/25,33.67,36705620.0000,34.05,34.25,33.620


In [0]:
satisfaccens.head(5)

,ID,State,Operating_System,CPU_Manufacturer,CPU_Year,Satisfaction,Budget Processor,OEM of GPU,What Year was your GPU released?,User's GPU Rating,User's Computer Rating
0,1,Tennessee,Linux,Intel,2011,Yes,Yes,AMD,2013,3,7
1,2,California,Classic Windows (Vista/XP/2000/ME/98),Other,2003,Yes,No,Other,2019,5,10
2,3,Delaware,MacOS,AMD,2019,No,Yes,Other,2001,2,1
3,4,Connecticut,Linux,Intel,2004,Yes,Yes,Intel (Integrated),2005,2,10
4,5,Alaska,Windows (10/8/7),Intel,2015,Yes,Yes,Nvidea,2014,5,7


In [0]:
searchqueries.head(5)

,Day Of July,AMD,Intel,Ryzen,Core I9
0,1,18,45,19,2
1,2,75,45,22,3
2,3,76,45,21,3
3,4,67,42,19,3
4,5,79,39,24,3


In [0]:
fig2 = go.Figure()
fig2.add_trace(go.Funnel(
    name = 'Intel',
    y = searchqueries['Day Of July'],
    x = searchqueries['Intel'],
    textinfo = "value+percent initial"))

fig2.add_trace(go.Funnel(
    name = 'AMD',
    orientation = "h",
    y= searchqueries['Day Of July'],
    x = searchqueries['AMD'],
    textposition = "inside",
    textinfo = "value+percent previous"))
fig2.update_layout(
    title=go.layout.Title(
        text="Amd vs Intel Google Search Queries, July 1-9, 2019",
        xref="paper",
        x=0,
        font=dict(
        family="Courier New, monospace",
        size=18,
        color="#FFB6C1"
            )
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Searches",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Day of July",
            font=dict(
                family="Courier New, monospace",
                size=25,
                color="#7f7f7f"
            )
        )
    )
)
fig2.show()

##Amd's Customer Satisfaction

In [0]:
satisfaccens.head(5)

,ID,State,Operating_System,CPU_Manufacturer,CPU_Year,Satisfaction,Budget Processor,OEM of GPU,What Year was your GPU released?,User's GPU Rating,User's Computer Rating
0,1,Tennessee,Linux,Intel,2011,Yes,Yes,AMD,2013,3,7
1,2,California,Classic Windows (Vista/XP/2000/ME/98),Other,2003,Yes,No,Other,2019,5,10
2,3,Delaware,MacOS,AMD,2019,No,Yes,Other,2001,2,1
3,4,Connecticut,Linux,Intel,2004,Yes,Yes,Intel (Integrated),2005,2,10
4,5,Alaska,Windows (10/8/7),Intel,2015,Yes,Yes,Nvidea,2014,5,7


In [0]:
satisfaccens["OEM of GPU"]= satisfaccens["OEM of GPU"].replace('Nvidea', "Nvidia")
fig = px.bar(satisfaccens, x='OEM of GPU', y="User's GPU Rating",
             labels={"User's GPU Rating":'User GPU Rating'}, height=400,
            title='GPU Rating by manufacturer')
fig.show()

In [0]:
nvidiadf = satisfaccens[satisfaccens['OEM of GPU'] == 'Nvidia']
amddf = satisfaccens[satisfaccens['OEM of GPU'] == 'AMD']
amddf.head(3)

,ID,State,Operating_System,CPU_Manufacturer,CPU_Year,Satisfaction,Budget Processor,OEM of GPU,What Year was your GPU released?,User's GPU Rating,User's Computer Rating
0,1,Tennessee,Linux,Intel,2011,Yes,Yes,AMD,2013,3,7
10,11,New Mexico,Windows (10/8/7),AMD,2018,Yes,No,AMD,2018,3,7
18,19,Iowa,Windows (10/8/7),AMD,2012,Yes,Yes,AMD,2012,4,9


In [0]:
amdaverage = em.find_mean(amddf["User's GPU Rating"])
nvidiaaverage = em.find_mean(nvidiadf["User's GPU Rating"])
print('Amd Average: ', amdaverage)
print('Nivida Average: ', nvidiaaverage)

Amd Average:  4.0
Nivida Average:  4.076923076923077


In [0]:
amddf = amddf.drop(columns=['ID', 'State','Operating_System','CPU_Manufacturer','CPU_Year','Satisfaction','Budget Processor','What Year was your GPU released?'])
amddf.head(5)

,OEM of GPU,User's GPU Rating,User's Computer Rating
0,AMD,3,7
10,AMD,3,7
18,AMD,4,9
26,AMD,5,7
27,AMD,3,6


In [0]:
amdpcaverage = em.find_mean(amddf["User's Computer Rating"])
nvidiapcaverage = em.find_mean(nvidiadf["User's Computer Rating"])
print(amdpcaverage)
print(nvidiapcaverage)

7.714285714285714
7.538461538461538


In [0]:
#The null is that the GPU OEM doesn't
#Factor into the User's Computer
#or GPU Rating
from scipy import stats
stats.ttest_ind(amddf["User's GPU Rating"], nvidiadf["User's GPU Rating"],
                 nan_policy='omit')
#Tvalue would appear to be pretty low, There is definitely a significant
#Difference between GPU OEMs

Ttest_indResult(statistic=-0.2106328492498221, pvalue=0.8355398445656304)

In [0]:
stats.ttest_ind(amddf["User's Computer Rating"], 
                nvidiadf["User's Computer Rating"],
                 nan_policy='omit')

Ttest_indResult(statistic=0.28736848324284026, pvalue=0.7771123303925562)

In [0]:
fig3 = px.bar(satisfaccens, x='OEM of GPU', y="User's Computer Rating",
             labels={"User's GPU Rating":'User GPU Rating'},
              title='Computer Rating compared to OEM of GPU',
              height=400)
fig3.show()

##Amd's Market Value over the past month

In [0]:
hist_earnamd.head(20)

,date,close,volume,open,high,low
0,16:00,30.45,"119,109,070",32.05,32.30,30.300
1,2019/07/30,33.87,72198880.0000,33.21,33.94,32.940
2,2019/07/29,33.48,45921980.0000,34.14,34.18,32.980
3,2019/07/26,34.02,43502780.0000,34.23,34.77,33.865
4,2019/07/25,33.67,36705620.0000,34.05,34.25,33.620
5,2019/07/24,34.11,41952260.0000,33.64,34.36,33.450
6,2019/07/23,33.49,38532460.0000,33.17,33.50,32.930
7,2019/07/22,32.85,37777310.0000,32.80,33.31,32.735
8,2019/07/19,32.51,44617700.0000,33.08,33.35,32.450
9,2019/07/18,33.00,58610860.0000,33.03,33.46,32.420


In [0]:
ig = px.line(hist_earnamd, x="date", y="volume", 
             title='AMD Stock Volume by Day, 2019')
ig.show()